## Data Wrangling: League of Legends Ranked Games

##### Import Module / Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
from pandas.io.json import json_normalize

##### Loading CSV files into dataframes

In [2]:
champs = 'champs.csv'
champs_df = pd.read_csv(champs)

duration = 'matches.csv'
duration_df = pd.read_csv(duration)

summoner_spell = 'summoner_spell_info.json'
summoner_spell_df = pd.read_json(summoner_spell)

participants = 'participants.csv'
participants_df = pd.read_csv(participants)

stats1 = 'stats1.csv'
stats1_df = pd.read_csv(stats1)

stats2 = 'stats2.csv'
stats2_df = pd.read_csv(stats2, low_memory=False)

bans = 'teambans.csv'
bans_df = pd.read_csv(bans)

macro_stats = 'games.csv'
macro_stats_df = pd.read_csv(macro_stats)

### Prepare Summoner Spell Dataframe

In [3]:
summoner_spell = pd.DataFrame(columns=['data']) # Prepare Dataframe from JSON
for row in summoner_spell_df.data:
    summoner_spell = summoner_spell.append(json_normalize(row))
summoner_spell.reset_index(inplace = True, drop = True)

summoner_spell = summoner_spell[['id','name']] # Select only neccessary columns

summoner_spell.id = summoner_spell.id.astype(int) # Convert id to whole numbers

summoner_spell = summoner_spell.rename(index = str, columns = {"name":'summoner_spell'}) # Rename column

summoner_spell.head(5) # Confirm changes

,id,summoner_spell
0,1,Cleanse
1,11,Smite
2,12,Teleport
3,13,Clarity
4,14,Ignite


### Prepare Duration Dataframe

In [4]:
duration_df.head(2)

,id,gameid,platformid,queueid,seasonid,duration,creation,version
0,10,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675
1,11,3187425281,EUW1,420,8,1693,1495066760778,7.10.187.9675


In [5]:
duration_df.duration = duration_df.duration * .01  # Convert duration to minutes

In [6]:
conditions = [ # Create list of current values
        duration_df.platformid == 'EUN1',
        duration_df.platformid == 'EUW1',
        duration_df.platformid == 'NA1',
        duration_df.platformid == 'TR1']
choices = ['Europe North', 'Europe West', 'North America', 'Turkey'] # Create list of new values

duration = duration_df[['platformid', 'seasonid', 'duration']].copy()

In [7]:
duration.platformid = np.select(conditions, choices) # Map conditions with choices
duration = duration.rename(index = str, columns = {"platformid": "Region","duration":"Duration","seasonid":'Season'}) #Rename columns

In [8]:
duration.head(2) # Confirm changes made

,Region,Season,Duration
0,Europe West,8,19.09
1,Europe West,8,16.93


In [9]:
path=r'/Users/Henry/Desktop/Springboard/Capstone Project/EDA' 
duration.to_csv(os.path.join(path,r'duration.csv')) # Export file to EDA folder

### Preparing Participants Dataframe

In [10]:
participants_df.head(2) # Explore dataframe

,id,matchid,player,championid,ss1,ss2,role,position
0,9,10,1,19,4,11,NONE,JUNGLE
1,10,10,2,267,3,4,DUO_SUPPORT,BOT


In [11]:
participants_merged = pd.merge(participants_df, # Merge champions dataframe with participants dataframe
                               champs_df,how='inner', left_on='championid'
                               , right_on='id').sort_values(by =['matchid'])

participants_columns = participants_merged[['matchid', # Select columns
                                            'player',
                                            'ss1',
                                            'ss2',
                                            'position',
                                            'name']] 

participants = participants_columns.rename(index = str, #Rename columns
                                           columns = {"matchid": "Match ID",
                                                      "player":"Player",
                                                      "position":"Position",
                                                      "name": 'Champion'}) 

participants_merged = pd.merge(participants, # Merge participants dataframe with summoner_spells dataframe
                               summoner_spell,how='inner', left_on='ss1'
                               , right_on='id').sort_values(by =['Match ID'])

participants_merged2 = pd.merge(participants_merged, # Merge participants dataframe with summoner_spells dataframe
                               summoner_spell,how='inner', left_on='ss2'
                               , right_on='id').sort_values(by =['Match ID'])

participants = participants.reset_index() # Reset index

participants = participants_merged2[['Match ID', 'Champion', 'summoner_spell_x', 'summoner_spell_y', "Position"]] # Select columns

participants = participants.rename(index = str, #Rename columns
                                           columns = {"summoner_spell_x": "SS1",
                                                      "summoner_spell_y":"SS2"}) 



In [32]:
participants.head(40) # Confirm preparation

,Match ID,Champion,SS1,SS2,Position
0,10,Warwick,Flash,Smite,JUNGLE
981535,10,Viktor,Flash,Exhaust,MID
1155620,10,Galio,Flash,Teleport,TOP
199705,10,Draven,Heal,Flash,BOT
199706,10,Jinx,Heal,Flash,BOT
199707,10,Fiora,Teleport,Flash,TOP
199708,10,Nami,Exhaust,Flash,BOT
199709,10,Ahri,Ignite,Flash,MID
199710,10,VelKoz,Ignite,Flash,BOT
199704,10,Skarner,Smite,Flash,JUNGLE


In [13]:
path=r'/Users/Henry/Desktop/Springboard/Capstone Project/EDA' 
participants.to_csv(os.path.join(path,r'participants_SS.csv')) # Export file to EDA folder

### Preparing Team Bans

In [14]:
bans_df.head(2)

,matchid,teamid,championid,banturn
0,10,100,11,1
1,10,100,117,3


In [15]:
bans_merged = pd.merge(bans_df, # Merge champions dataframe with bans dataframe
                               champs_df,how='inner', left_on='championid'
                               , right_on='id').sort_values(by =['matchid'])
bans_columns = bans_merged[['matchid', # Select columns
                                            'banturn',
                                            'name']] 
bans = bans_columns.rename(index = str, #Rename columns
                                           columns = {"matchid": "Match ID",
                                                      "banturn":"Ban Turn",
                                                      "name": 'Champion'})

In [16]:
bans = bans.reset_index() # Reset index
bans = bans[['Match ID', 'Ban Turn', 'Champion']] # Reselect columns after resetting index

In [17]:
bans.head(2)

,Match ID,Ban Turn,Champion
0,10,1,Master Yi
1,10,6,Fizz


In [18]:
path=r'/Users/Henry/Desktop/Springboard/Capstone Project/EDA' 
bans.to_csv(os.path.join(path,r'bans.csv')) # Export file to EDA folder

### Preparing Micro Stats

In [30]:
stats1_df.head(100) # Explore stats1 dataframe

,id,win,item1,item2,item3,item4,item5,item6,trinket,kills,...,neutralminionskilled,ownjunglekills,enemyjunglekills,totcctimedealt,champlvl,pinksbought,wardsbought,wardsplaced,wardskilled,firstblood
0,9,0,3748,2003,3111,3053,1419,1042,3340,6,...,69,42,27,610,13,0,0,10,0,0
1,10,0,2301,3111,3190,3107,0,0,3364,0,...,1,1,0,211,14,1,0,17,3,0
2,11,0,1055,3072,3006,3031,3046,1036,3340,7,...,3,1,2,182,14,1,0,13,5,0
3,12,0,1029,3078,3156,1001,3053,0,3340,5,...,6,6,0,106,15,0,0,3,0,0
4,13,0,3020,1058,3198,3102,1052,1026,3340,2,...,4,3,1,159,15,0,0,10,0,0
5,14,1,1400,3111,3078,3742,1033,3067,3340,3,...,111,81,30,808,16,0,0,8,2,0
6,15,1,3025,3193,3068,3047,1028,3082,3363,4,...,6,0,6,179,16,1,0,7,0,0
7,16,1,3135,3165,3089,3020,1058,3136,3340,13,...,6,3,3,48,17,0,0,8,0,0
8,17,1,3046,3031,3006,3085,3072,1029,3340,15,...,7,2,5,440,16,1,0,6,1,1
9,18,1,1058,2045,3092,3089,3113,3158,3364,4,...,2,1,1,260,16,5,0,25,7,0


In [20]:
stats2_df.head(2) # Explore stats2 dataframe

,id,win,item1,item2,item3,item4,item5,item6,trinket,kills,...,neutralminionskilled,ownjunglekills,enemyjunglekills,totcctimedealt,champlvl,pinksbought,wardsbought,wardsplaced,wardskilled,firstblood
0,1028382,0,1056,3001,1052,3020,1058,1026,3340,7,...,0,0,0,50,12,0,0,6,0,0
1,1028383,0,1041,2003,0,0,0,0,3340,0,...,11,11,0,114,3,0,0,0,0,0


In [21]:
stats2_df.columns

Index(['id', 'win', 'item1', 'item2', 'item3', 'item4', 'item5', 'item6',
       'trinket', 'kills', 'deaths', 'assists', 'largestkillingspree',
       'largestmultikill', 'killingsprees', 'longesttimespentliving',
       'doublekills', 'triplekills', 'quadrakills', 'pentakills',
       'legendarykills', 'totdmgdealt', 'magicdmgdealt', 'physicaldmgdealt',
       'truedmgdealt', 'largestcrit', 'totdmgtochamp', 'magicdmgtochamp',
       'physdmgtochamp', 'truedmgtochamp', 'totheal', 'totunitshealed',
       'dmgselfmit', 'dmgtoobj', 'dmgtoturrets', 'visionscore', 'timecc',
       'totdmgtaken', 'magicdmgtaken', 'physdmgtaken', 'truedmgtaken',
       'goldearned', 'goldspent', 'turretkills', 'inhibkills',
       'totminionskilled', 'neutralminionskilled', 'ownjunglekills',
       'enemyjunglekills', 'totcctimedealt', 'champlvl', 'pinksbought',
       'wardsbought', 'wardsplaced', 'wardskilled', 'firstblood'],
      dtype='object')

In [22]:
stats = stats1_df.append([stats2_df]) # Union stats dataframes
stats.shape # Confirm union

(1834517, 56)

In [23]:
stats = stats[['id', 'win', 'kills', # Select Columns
               'neutralminionskilled','ownjunglekills','enemyjunglekills',
               'totcctimedealt', 'champlvl', 'pinksbought','wardsbought','wardsplaced','wardskilled','firstblood']]

In [24]:
stats.head(2)

,id,win,kills,neutralminionskilled,ownjunglekills,enemyjunglekills,totcctimedealt,champlvl,pinksbought,wardsbought,wardsplaced,wardskilled,firstblood
0,9,0,6,69,42,27,610,13,0,0,10,0,0
1,10,0,0,1,1,0,211,14,1,0,17,3,0


In [25]:
path=r'/Users/Henry/Desktop/Springboard/Capstone Project/EDA' 
stats.to_csv(os.path.join(path,r'micro_stats.csv')) # Export file to EDA folder

### Preparing Macro Stats

In [26]:
macro_stats_df.columns # Explore Dataframe

Index(['gameId', 'creationTime', 'gameDuration', 'seasonId', 'winner',
       'firstBlood', 'firstTower', 'firstInhibitor', 'firstBaron',
       'firstDragon', 'firstRiftHerald', 't1_champ1id', 't1_champ1_sum1',
       't1_champ1_sum2', 't1_champ2id', 't1_champ2_sum1', 't1_champ2_sum2',
       't1_champ3id', 't1_champ3_sum1', 't1_champ3_sum2', 't1_champ4id',
       't1_champ4_sum1', 't1_champ4_sum2', 't1_champ5id', 't1_champ5_sum1',
       't1_champ5_sum2', 't1_towerKills', 't1_inhibitorKills', 't1_baronKills',
       't1_dragonKills', 't1_riftHeraldKills', 't1_ban1', 't1_ban2', 't1_ban3',
       't1_ban4', 't1_ban5', 't2_champ1id', 't2_champ1_sum1', 't2_champ1_sum2',
       't2_champ2id', 't2_champ2_sum1', 't2_champ2_sum2', 't2_champ3id',
       't2_champ3_sum1', 't2_champ3_sum2', 't2_champ4id', 't2_champ4_sum1',
       't2_champ4_sum2', 't2_champ5id', 't2_champ5_sum1', 't2_champ5_sum2',
       't2_towerKills', 't2_inhibitorKills', 't2_baronKills', 't2_dragonKills',
       't2_riftHer

In [27]:
columns = ['t1_towerKills','t1_inhibitorKills','t1_dragonKills','t1_baronKills', # Select potential variables that can predict the outcome of a game
                  't2_towerKills','t2_inhibitorKills','t2_dragonKills','t2_baronKills',
                       'firstBlood', 'firstTower', 'firstInhibitor', 
                           'firstBaron','firstDragon', 'firstRiftHerald',
                              'winner'] 

In [28]:
macro_stats_df = macro_stats_df[columns] # Select columns
macro_stats_df.head() # Confirm changes

,t1_towerKills,t1_inhibitorKills,t1_dragonKills,t1_baronKills,t2_towerKills,t2_inhibitorKills,t2_dragonKills,t2_baronKills,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,winner
0,11,1,3,2,5,0,1,0,2,1,1,1,1,2,1
1,10,4,2,0,2,0,0,0,1,1,1,0,1,1,1
2,8,1,1,1,2,0,1,0,2,1,1,1,2,0,1
3,9,2,2,1,0,0,0,0,1,1,1,1,1,0,1
4,9,2,3,1,3,0,1,0,2,1,1,1,1,0,1


In [29]:
path=r'/Users/Henry/Desktop/Springboard/Capstone Project/EDA' 
macro_stats_df.to_csv(os.path.join(path,r'macro_stats.csv')) # Export file to EDA folder